# STRT 6601 Applied Economics & Modeling
Problem Set 1

**SAXA 3**

Mike Johnson | Kris Lederer | Sebastian Martinez | Ryan Mathis | Khushi Patel
****

## Set up

In [12]:
# Import libraries
import pandas as pd
from linearmodels import IV2SLS

In [2]:
# Load data
df = pd.read_csv('STRT601-ProblemSet1-DataSet-ae98v2.csv')

df

,momweeksworked,kidcount,samesex,twins2,momworked,morekids,moreths,blackm,hispm,whitem
0,0,2,0,0,0,0,0,0,0,1
1,52,2,0,0,1,0,0,0,0,1
2,30,2,0,0,1,0,0,0,0,1
3,0,2,0,0,0,0,0,0,0,1
4,0,2,0,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...
402009,0,3,0,0,0,1,0,0,0,1
402010,36,3,1,0,1,1,1,0,0,1
402011,40,2,1,0,1,0,1,0,0,1
402012,16,2,0,0,1,0,1,0,0,1


****

## 1. Provide a table with summary statistics for the relevant variables.

In [3]:
# Create table with summary statistics
summary_stats = df.describe().T

summary_stats

,count,mean,std,min,25%,50%,75%,max
momweeksworked,402014.0,20.823718,22.281936,0.0,0.0,12.0,49.0,52.0
kidcount,402014.0,2.559433,0.816565,2.0,2.0,2.0,3.0,12.0
samesex,402014.0,0.505448,0.499971,0.0,0.0,1.0,1.0,1.0
twins2,402014.0,0.008522,0.091921,0.0,0.0,0.0,0.0,1.0
momworked,402014.0,0.565455,0.495698,0.0,0.0,1.0,1.0,1.0
morekids,402014.0,0.405120,0.490916,0.0,0.0,0.0,1.0,1.0
moreths,402014.0,0.291987,0.454677,0.0,0.0,0.0,1.0,1.0
blackm,402014.0,0.123779,0.329330,0.0,0.0,0.0,0.0,1.0
hispm,402014.0,0.030531,0.172044,0.0,0.0,0.0,0.0,1.0
whitem,402014.0,0.817016,0.386654,0.0,1.0,1.0,1.0,1.0


****

## 2. Suppose one were to estimate equation 1 using OLS.

### 2.1. Explain the conditions under which OLS provides unbiased and consistent estimates of the causal effect.

For OLS to provide unbiased and consistent estimates of the causal effect, the regressor of interest (i.e., `morekids`) must be uncorrelated with the error term (exogenous). 

When this condition holds:
* **Unbiased**: On average, the OLS estimate equals $𝛽_1$
* **Consistent**: As the number of observations increases, the estimate converges to $𝛽_1$

### 2.2. Do you think these conditions likely to hold? Why or why not?

No, because the decision to have more children (`morekids`) is likely endogenous because it's correlated with unobserved factors that could also impact the mother's decision. This creates omitted variable bias.

Potential sources of edogeneity:
* **Career Preferences**: Women who prefer having more childen might also have different preferences with work-life balance or career priorities.
* **Family Conditions**: Family weatlh, child support, or health could influence the decision to have more children or work.
* **Economic Conditions**: The cost of living and the economic opportunities available could influce the decision to have more children or work.

Since these unobserved facters likely correlate to the decision to have more childen, the exogeneity assumption is violated.

****

## 3. Angrist and Evans propose that $x_i$ is an endogenous regressor, and that `samesex` and `twins2` can be used as instrumental variables in a 2SLS regression. Explain why these instruments might satisfy the relevance and orthogonality conditions.

### Relevance

`samesex` (i.e., the first two children are the same sex): Parents may be inclined to try to have a child of the opposite sex.

In [15]:
# This has cleaner syntax for multiple variables
iv_samesex = IV2SLS(
    dependent=df['momworked'],
    exog=df[['moreths', 'blackm', 'hispm', 'whitem']],
    endog=df[['morekids']], 
    instruments=df[['samesex']]
)
results_samesex = iv_samesex.fit()
print(results_samesex.summary)  # Note: no parentheses for linearmodels

                          IV-2SLS Estimation Summary                          
Dep. Variable:              momworked   R-squared:                      0.5035
Estimator:                    IV-2SLS   Adj. R-squared:                 0.5035
No. Observations:              402014   F-statistic:                  4.62e+05
Date:                Tue, Sep 09 2025   P-value (F-stat)                0.0000
Time:                        00:18:42   Distribution:                  chi2(5)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
moreths        0.1185     0.0023     50.593     0.0000      0.1139      0.1231
blackm         0.4943     0.0107     45.985     0.00

In [16]:
# Check first-stage F-statistic
first_stage_f = results_samesex.first_stage.diagnostics['f_stat'].iloc[0]
print(f"First-stage F-statistic: {first_stage_f:.4f}")
print(f"Relevance test (F > 10): {'PASS' if first_stage_f > 10 else 'FAIL'}")

KeyError: 'f_stat'

`twins2`(i.e., twins on the second pregnancy): Having twins on the second pregnancy automatically increases the family size from 2 to 3.